<a href="https://colab.research.google.com/github/merliou/term_paper_css/blob/main/bert_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# Installiere die notwendigen Bibliotheken
!pip install bertopic
!pip install umap-learn
!pip install pandas
!pip install numpy

# Importiere die notwendigen Bibliotheken
import pandas as pd
import numpy as np
from bertopic import BERTopic
from umap import UMAP
#from hdbscan import HDBSCAN

# Lade die CSV-Datei
data = pd.read_csv("10_data_covs_avg_no_doubles.csv", header=0)

# draw 10 percent random sample
np.random.seed(161)
data = data.sample(frac=0.1)

data['ort'] = data['ort']
data['bundesland'] = data['bundesland']
data['rating_rez'] = data['bewertung']
data['rezension'] = data['rezension']
data['traeger'] = data['Traeger']
data['typ'] = data['EinrichtungsTyp']
data['versorgungsstufe'] = data['Allgemeine_Notfallversorgung']
data['bettenzahl'] = data['INSG']
data['rating_avg'] = data['rating']
data['number_of_ratings'] = data['total_ratings']

data['bundesland'].value_counts()
data['rating_rez'].value_counts()
data['traeger'].value_counts()
data['typ'].value_counts()
data['versorgungsstufe'].value_counts()
data['rating_avg'].value_counts()


# Zeige die ersten 3 Zeilen der Daten an
print(data.iloc[0:3])

## Bereite die Daten vor (falls nötig)
## In diesem Fall verwenden wir die Spalte 'rezension' für die Textanalyse
#documents = data['rezension'].tolist()
#
## Erstelle das UMAP-Modell
#umap_model = UMAP(random_state=1337)
#
## Erstelle das BERTopic-Modell
#topic_model = BERTopic(umap_model=umap_model)
#
## Führe das Modell auf den Dokumenten aus
#topics, probs = topic_model.fit_transform(documents)
#
## Zeige die häufigsten Themen an
#topic_info = topic_model.get_topic_info()
#print(topic_info)

,count
rating_rez,
5,7135
1,3054
4,741
2,473
3,379


Data cleaning

In [ ]:
# Überprüfe auf fehlende Werte in der Spalte 'rezension'
print(data['rezension'].isnull().sum())

# Entferne Zeilen mit fehlenden Werten
data = data.dropna(subset=['rezension'])

# Oder ersetze fehlende Werte durch einen leeren String
data['rezension'] = data['rezension'].fillna("")

# Extrahiere die Dokumente
documents = data['rezension'].tolist()

# Überprüfe die ersten 5 Dokumente
print(documents[:5])

# Überprüfe den Datentyp der Spalte 'rezension'
print(data['rezension'].dtype)

data['rezension'] = data['rezension'].astype(str)

0


Modell erneut ausführen

In [ ]:
# Extrahiere die Dokumente
documents = data['rezension'].tolist()

# Erstelle das BERTopic-Modell
topic_model = BERTopic()

# Führe das Modell auf den Dokumenten aus
topics, probs = topic_model.fit_transform(documents)

# Zeige die Themeninformationen an
topic_info = topic_model.get_topic_info()
print(topic_info)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

    Topic  Count                                             Name  \
0      -1   6722                               -1_und_ich_die_der   
1       0    503                             0_klinik_die_und_der   
2       1    317                             1_klinik_die_in_sehr   
3       2    289                           2_station_auf_sehr_war   
4       3    262                              3_die_und_nicht_ich   
..    ...    ...                                              ...   
72     71     10  71_wartezeit_wartezeiten_wartezimmer_behandlung   
73     72     10               72_im_arbeitsgruppen_1x_wchentlich   
74     73     10                   73_toll_achern_drbrunswig_team   
75     74     10          74_schmerzen_schmerzmittel_starken_raus   
76     75     10                  75_klinik_bavaria_mic_karlsruhe   

                                       Representation  \
0   [und, ich, die, der, das, in, war, sehr, nicht...   
1   [klinik, die, und, der, in, ich, ist, zu, das,...   


In [ ]:
# Zeige die Dokumentinformationen an
document_info = topic_model.get_document_info(documents)
print(document_info)

# Visualisiere die Ergebnisse
# Themen-Distanz
topic_model.visualize_topics()

                                                Document  Topic  \
0      Von A-Z positiv. Teil 1\nVom Reinigungspersona...     -1   
1      Ich kann die vielen schlechten Rezensionen übe...     -1   
2      Ich bin total begeistert von dem Personal  auf...      2   
3      Enttäuschende Erfahrung in der Notaufnahme\n\n...     66   
4      Das Personal in der Aufnahme ist überfordert u...      8   
...                                                  ...    ...   
11627  Meine 81 Jährige Mutter wurde auf Station 24 b...     36   
11628  Absolut zufrieden mit dem Service und vorallem...     -1   
11629  Ich kann jetzt nur für Kreißsaal sowie Station...     -1   
11630  Durch das schnelle und kompetente Handeln alle...      4   
11631  Vielen lieben Dank an das gesamte Team der Sta...     -1   

                                            Name  \
0                             -1_und_ich_die_der   
1                             -1_und_ich_die_der   
2                         2_station_auf

In [ ]:
# Wort-Scores
topic_model.visualize_barchart()

In [ ]:
# Themen-Similarität
topic_model.visualize_heatmap()

In [ ]:
# Themen-Hierarchie
topic_model.visualize_hierarchy()

In [ ]:
# Zeige spezifische Themen an (z.B. Thema 49)
specific_topic = topic_model.get_topic(49)
print(specific_topic)

# Zeige repräsentative Dokumente für spezifische Themen an (z.B. Thema 49 und 82)
representative_docs_49 = topic_model.get_representative_docs(topic=49)
representative_docs_82 = topic_model.get_representative_docs(topic=82)
print(representative_docs_49)
print(representative_docs_82)

[('station', np.float64(0.03075882698078743)), ('klinik', np.float64(0.027546367922963364)), ('bis', np.float64(0.0220624117142804)), ('31', np.float64(0.019609307870642197)), ('sehr', np.float64(0.019021210113480475)), ('vom', np.float64(0.017089858739553477)), ('mich', np.float64(0.015538967779543769)), ('an', np.float64(0.015220149515462798)), ('stationr', np.float64(0.014620596392466096)), ('ihr', np.float64(0.013943521142597984))]
['Eine sehr gute Klinik. Ich war 1 Woche stationär auf St. 31 und wurde sehr gut betreut. Ein großes Lob und Dankeschön an Dr. M. Stockbauer und OÄ Dr. Wallisch für die hervorragende Behandlung.\nAlle Mitarbeiter/innen auf St. 31 geben ihr bestes und waren überaus freundlich und bemüht.\nDer einzigste Minuspunkt ist das Essen, leider nicht sehr gut, aber auch das überlebt man :-)\nViele Grüße,  Claudia Grimm', 'Ich war vor einer Woche in der Klinik. Angefangen bei der Patientenaufnahme über das OP-Team bis zur Station sehr freundliche und hilfsbereite Me

In [ ]:
# Benutzerdefinierte Themen-Labels setzen
topic_model.set_topic_labels({49: "Krankenhaus Erfahrungen", 82: "Psychiatrische Behandlung"})
print(topic_model.get_topic_info(49))
print(topic_model.get_topic_info(82))

# Themen pro Klasse (falls eine Klassenspalte vorhanden ist)
# In diesem Fall verwenden wir die Spalte 'bewertung' als Klasse
classes = data['bewertung']

topics_per_class = topic_model.topics_per_class(documents, classes=classes)
topic_model.visualize_topics_per_class(topics_per_class, top_n_topics=31)

# Ändere das Clustering-Modell zu KMeans
from sklearn.cluster import KMeans

cluster_model = KMeans(n_clusters=100)  # Verwende KMeans-Clustering mit 100 Themen
topic_model_kmeans = BERTopic(hdbscan_model=cluster_model)
topicsKmeans, probsKmeans = topic_model_kmeans.fit_transform(documents)

# Zeige die Themeninformationen für das KMeans-Modell an
topic_info_kmeans = topic_model_kmeans.get_topic_info()
print(topic_info_kmeans)

# Zeige die Dokumentinformationen für das KMeans-Modell an
document_info_kmeans = topic_model_kmeans.get_document_info(documents)
print(document_info_kmeans)

## Geführte Themenmodelle (Guided Topic Models)
#seed_topic_list = [["krankenhaus", "arzt", "pflege"],
#                   ["psychiatrie", "therapie", "depression"],
#                   ["operation", "schmerz", "reha"],
#                   ["notaufnahme", "wartezeit", "behandlung"]]

topic_model_guided = BERTopic(seed_topic_list=seed_topic_list)
topics_guided, probs_guided = topic_model_guided.fit_transform(documents)

# Zeige die Themeninformationen für das geführte Modell an
topic_info_guided = topic_model_guided.get_topic_info()
print(topic_info_guided)

# Visualisiere die Ergebnisse des geführten Modells
topic_model_guided.visualize_barchart(n_words=8, top_n_topics=12)

# Zeige die Dokumentinformationen für das geführte Modell an
document_info_guided = topic_model_guided.get_document_info(documents)
print(document_info_guided)

[('leider', np.float64(0.07549936136309979)), ('wir', np.float64(0.01994208869630371)), ('bauweise', np.float64(0.017715348402315312)), ('ist', np.float64(0.017458316938083175)), ('faden', np.float64(0.015908440684974542)), ('der', np.float64(0.014445241033167027)), ('sehr', np.float64(0.013822813159544534)), ('mal', np.float64(0.013768958548855573)), ('nicht', np.float64(0.012958368784570804)), ('liegt', np.float64(0.011912065392922206))]
['Dies ist ein großes Krankenhaus,  welches in stalinistischer Bauweise auf einem großen Gelände liegt.  Diese Bauweise ist in Eisenhüttenstadt sehr präsent . Leider sehr Energie-intensiv .\nRiesige Flure und Treppen lassen die Heizungszähler in Hochgeschwindigkeit laufen 🏃\u200d♂️ 🏃\u200d♂️  , was für den Geldbeutel  sehr schlecht ist  .\nDas Personal ist sehr freundlich und hilfsbereit.  Leider ist von Ihnen zu wenig vorhanden  , denn trotz Termin wartet man für manche Leistungen auch mal 2,5 - 3 Stunden .\nAnsonsten ist es sauber und ich habe mich